In [72]:
import pandas as pd
import re
import spacy
import textacy
import glob
import visualise_spacy_tree
from spacy.matcher import Matcher 
import os
import sys
import en_core_web_trf
import spacy_transformers
from pathlib import Path
from pprint import pprint
from nltk import pos_tag
from sklearn_crfsuite import CRF
from sklearn.metrics import confusion_matrix

In [33]:
sys.executable

'/Users/lbeato/anaconda3/bin/python'

In [34]:
!"/Users/lbeato/anaconda3/bin/python"  -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [35]:
# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

In [36]:
pd.set_option('display.max_colwidth', 2000)
pd.options.display.max_rows=100

### Cargando los discurso de Rep. Dom. a un Dataframe

In [38]:
# Ruta de la carpeta
folders = glob.glob('UNGDC_1946-2022/Session*')


# Dataframe
df = pd.DataFrame(columns=['Pais','Discurso','Sesion','Anio'])

# Leyendo los discursos de Rep Dominicana
i = 0 
for file in folders:

    
    speech = glob.glob(file+'/DOM*.txt')
    
    if (len(speech) > 0):

        with open(speech[0],encoding='utf8') as f:
            # Discurso
            df.loc[i,'Discurso'] = f.read()
            # Anio 
            df.loc[i,'Anio'] = speech[0].split('_')[-1].split('.')[0]
            # Sesion
            df.loc[i,'Sesion'] = speech[0].split('_')[-2]
            # Pais
            df.loc[i,'Pais'] = speech[0].split('_')[0].split("\\")[-1]
      
            i += 1 

In [39]:
df

,Pais,Discurso,Sesion,Anio
0,UNGDC,"Had the United Nations not been faced in its five years of existence with countless obstacles, had it not been called upon to deal with so many alarming symptoms in the historical development of a convalescent world, it would be difficult to justify its creation, to welcome its birth and to hang on it, as men hoist a banner to a masthead, the hopes of peace and security of a profoundly disturbed epoch. The fact is that the struggle which this young yet already sorely tried Organization has waged, and is still waging, is the best proof of the timeliness of its conception and creation, and of the urgent and imperious need for the work it is doing today.\n2.\tCertainly this has not been merely another episode in the political history of the peoples. The recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement. That is why the United Nations (united for agreement, concord and peace) neither can nor should be regarded as one of those familiar panaceas created for interested motives in international political laboratories with the object of providing temporary compromise solutions for more or less selfish problems; it must be recognized for what it is, a sacred idea conceived by the peoples at a time of grave and legitimate struggle for the permanent protection of their democratic existence, autonomy, dignity, peace and security.\n3.\tFor these reasons it is a crime to obstruct it in its sacred mission. It is a crime to try to reduce it to a tool of selfish ideologies. It is a crime to use it as an instrument of war in times of peace. It is a crime not to render it due honour.\n4.\t“We the peoples of the United Nations, determined to save succeeding generations from the scourge of war…” Thus reads the opening sentence of the Preamble of the United Nations Charter. Yet, today one of these nations, “determined to save succeeding generations from the scourge of war” is viol...",05,1950
1,UNGDC,"At the outset, I would like to congratulate Ambassador María Fernanda Espinosa Garcés on her election as President of the General Assembly and to wish her every possible success. It is a great honour for me to be able to once again address the Assembly as President of the Dominican Republic. This is a special occasion for our country, because next year the Dominican Republic will sit on the Security Council as a non-permanent member for the first time.\nOn behalf of the Dominican people and myself, I would therefore like to start by thanking the States Members of the United Nations for the support we have received in joining the Security Council. In particular, I thank the members of the Group of Latin American and Caribbean States for the trust placed in us. Our purpose during our term will be to contribute a voice for Latin America and the Caribbean to one of the forums for greater global responsibility in the maintenance of peace.\nOf course, we are committed to asserting, both in the Security Council and in the Assembly, the same unbreakable principles that guide us as a nation. Respect for human rights, democracy, freedom and diversity will therefore be the beacon that we will follow in order to guide our work in the Security Council. We will do everything possible to honour the trust placed in us, being fully aware that, although fragile, stability and security are priority objectives that must be preserved and progress made towards, step by step and day by day.\nIt is clear to us that in a space that addresses issues that can change the course of history and the fate of millions of people, we cannot seek easy and drastic solutions, which in any case do not exist. In that context, our country will always be a place of dialogue\n \nand understanding, and will always seek peaceful solutions. We are always willing to do whatever it takes to find common ground and support consensus as the best tool for peace an

### Preprocesamiento del texto

#### Funcion para limpiar el texto

In [40]:
def limpiar(text):
    # Quitar numero de parrafo
    text = re.sub('[0-9]+.\t', '', str(text))
    text = re.sub('[0-9]+.', '', str(text))
    # Quitando algunos caracteres
    text = re.sub('\n', '', str(text))
    text = re.sub('\n ', '', str(text))
    text = re.sub('-', '', str(text))
    text = re.sub('- ', '', str(text))
    text = re.sub('"', '', str(text))
    text = re.sub("'s", '', str(text))
    
    return text
    

In [41]:
df['Discurso_Limpio']=df['Discurso'].apply(limpiar)

In [42]:
df

,Pais,Discurso,Sesion,Anio,Discurso_Limpio
0,UNGDC,"Had the United Nations not been faced in its five years of existence with countless obstacles, had it not been called upon to deal with so many alarming symptoms in the historical development of a convalescent world, it would be difficult to justify its creation, to welcome its birth and to hang on it, as men hoist a banner to a masthead, the hopes of peace and security of a profoundly disturbed epoch. The fact is that the struggle which this young yet already sorely tried Organization has waged, and is still waging, is the best proof of the timeliness of its conception and creation, and of the urgent and imperious need for the work it is doing today.\n2.\tCertainly this has not been merely another episode in the political history of the peoples. The recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement. That is why the United Nations (united for agreement, concord and peace) neither can nor should be regarded as one of those familiar panaceas created for interested motives in international political laboratories with the object of providing temporary compromise solutions for more or less selfish problems; it must be recognized for what it is, a sacred idea conceived by the peoples at a time of grave and legitimate struggle for the permanent protection of their democratic existence, autonomy, dignity, peace and security.\n3.\tFor these reasons it is a crime to obstruct it in its sacred mission. It is a crime to try to reduce it to a tool of selfish ideologies. It is a crime to use it as an instrument of war in times of peace. It is a crime not to render it due honour.\n4.\t“We the peoples of the United Nations, determined to save succeeding generations from the scourge of war…” Thus reads the opening sentence of the Preamble of the United Nations Charter. Yet, today one of these nations, “determined to save succeeding generations from the scourge of war” is viol...",05,1950,"Had the United Nations not been faced in its five years of existence with countless obstacles, had it not been called upon to deal with so many alarming symptoms in the historical development of a convalescent world, it would be difficult to justify its creation, to welcome its birth and to hang on it, as men hoist a banner to a masthead, the hopes of peace and security of a profoundly disturbed epoch. The fact is that the struggle which this young yet already sorely tried Organization has waged, and is still waging, is the best proof of the timeliness of its conception and creation, and of the urgent and imperious need for the work it is doing today.Certainly this has not been merely another episode in the political history of the peoples. The recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement. That is why the United Nations (united for agreement, concord and peace) neither can nor should be regarded as one of those familiar panaceas created for interested motives in international political laboratories with the object of providing temporary compromise solutions for more or less selfish problems; it must be recognized for what it is, a sacred idea conceived by the peoples at a time of grave and legitimate struggle for the permanent protection of their democratic existence, autonomy, dignity, peace and security.For these reasons it is a crime to obstruct it in its sacred mission. It is a crime to try to reduce it to a tool of selfish ideologies. It is a crime to use it as an instrument of war in times of peace. It is a crime not to render it due honour.“We the peoples of the United Nations, determined to save succeeding generations from the scourge of war…” Thus reads the opening sentence of the Preamble of the United Nations Charter. Yet, today one of these nations, “determined to save succeeding gene

#### Separar el texto en oraciones

In [43]:
def oraciones(text):
    oracion_limpia=[]
    text = re.split('[.?]', text)
    for sent in text:
        oracion_limpia.append(sent)
    return oracion_limpia    
    

In [44]:
df['Oraciones']=df['Discurso_Limpio'].apply(oraciones)

In [45]:
df

,Pais,Discurso,Sesion,Anio,Discurso_Limpio,Oraciones
0,UNGDC,"Had the United Nations not been faced in its five years of existence with countless obstacles, had it not been called upon to deal with so many alarming symptoms in the historical development of a convalescent world, it would be difficult to justify its creation, to welcome its birth and to hang on it, as men hoist a banner to a masthead, the hopes of peace and security of a profoundly disturbed epoch. The fact is that the struggle which this young yet already sorely tried Organization has waged, and is still waging, is the best proof of the timeliness of its conception and creation, and of the urgent and imperious need for the work it is doing today.\n2.\tCertainly this has not been merely another episode in the political history of the peoples. The recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement. That is why the United Nations (united for agreement, concord and peace) neither can nor should be regarded as one of those familiar panaceas created for interested motives in international political laboratories with the object of providing temporary compromise solutions for more or less selfish problems; it must be recognized for what it is, a sacred idea conceived by the peoples at a time of grave and legitimate struggle for the permanent protection of their democratic existence, autonomy, dignity, peace and security.\n3.\tFor these reasons it is a crime to obstruct it in its sacred mission. It is a crime to try to reduce it to a tool of selfish ideologies. It is a crime to use it as an instrument of war in times of peace. It is a crime not to render it due honour.\n4.\t“We the peoples of the United Nations, determined to save succeeding generations from the scourge of war…” Thus reads the opening sentence of the Preamble of the United Nations Charter. Yet, today one of these nations, “determined to save succeeding generations from the scourge of war” is viol...",05,1950,"Had the United Nations not been faced in its five years of existence with countless obstacles, had it not been called upon to deal with so many alarming symptoms in the historical development of a convalescent world, it would be difficult to justify its creation, to welcome its birth and to hang on it, as men hoist a banner to a masthead, the hopes of peace and security of a profoundly disturbed epoch. The fact is that the struggle which this young yet already sorely tried Organization has waged, and is still waging, is the best proof of the timeliness of its conception and creation, and of the urgent and imperious need for the work it is doing today.Certainly this has not been merely another episode in the political history of the peoples. The recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement. That is why the United Nations (united for agreement, concord and peace) neither can nor should be regarded as one of those familiar panaceas created for interested motives in international political laboratories with the object of providing temporary compromise solutions for more or less selfish problems; it must be recognized for what it is, a sacred idea conceived by the peoples at a time of grave and legitimate struggle for the permanent protection of their democratic existence, autonomy, dignity, peace and security.For these reasons it is a crime to obstruct it in its sacred mission. It is a crime to try to reduce it to a tool of selfish ideologies. It is a crime to use it as an instrument of war in times of peace. It is a crime not to render it due honour.“We the peoples of the United Nations, determined to save succeeding generations from the scourge of war…” Thus reads the opening sentence of the Preamble of the United Nations Charter. Yet, today one of these nations, “determined to save succe

In [47]:
# Creando un dataframe nuevo en donde tengo las diferentes oraciones por anio y el tamano que tiene cada una

df2 = pd.DataFrame(columns=['Oracion','Anio','Longitud'])

lista_oraciones=[]

for i in range(len(df)):
    for sent in df.loc[i,'Oraciones']:
        cantidad_palabras=len(sent.split())
        anio=df.loc[i,'Anio']
        diccionario={'Oracion':sent,'Anio':anio,'Longitud':cantidad_palabras}
        lista_oraciones.append(diccionario)
        
df2 = pd.DataFrame(lista_oraciones)
        


In [48]:
df2

,Oracion,Anio,Longitud
0,"Had the United Nations not been faced in its five years of existence with countless obstacles, had it not been called upon to deal with so many alarming symptoms in the historical development of a convalescent world, it would be difficult to justify its creation, to welcome its birth and to hang on it, as men hoist a banner to a masthead, the hopes of peace and security of a profoundly disturbed epoch",1950,73
1,"The fact is that the struggle which this young yet already sorely tried Organization has waged, and is still waging, is the best proof of the timeliness of its conception and creation, and of the urgent and imperious need for the work it is doing today",1950,46
2,Certainly this has not been merely another episode in the political history of the peoples,1950,15
3,"The recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement",1950,23
4,"That is why the United Nations (united for agreement, concord and peace) neither can nor should be regarded as one of those familiar panaceas created for interested motives in international political laboratories with the object of providing temporary compromise solutions for more or less selfish problems; it must be recognized for what it is, a sacred idea conceived by the peoples at a time of grave and legitimate struggle for the permanent protection of their democratic existence, autonomy, dignity, peace and security",1950,82
...,...,...,...
5651,We find that the people are free,2014,7
5652,They are free to know which parts of their past they wish to use as a guide to their future and which they wish to leave aside,2014,27
5653,"We will find a way to work together, as we have done with the neighbours with whom we share an island, or as we found yesterday, during the climate summit, with the other countries with which we share the planet",2014,40
5654,"The citizens that we represent, increasingly better informed, demand that we live up to our responsibility and our declared commitment: to preserve peace, to promote development and to make education and health care the right of all inhabitants of the planet",2014,41


In [49]:
count=0
for i in range(len(df2)):
    if df2.loc[i,'Longitud']==0:
        count+=1
print(count)    

82


### Tarea 1: Extraccion de Iniciativas desde los discursos

In [50]:
def oracion_iniciativa(text):
    patrones=[
               r'\b'+'conference'+r'\b',  
               r'\b'+'plan'+r'\b', 
               r'\b'+'agreement'+r'\b', 
            ]
    
    bandera=0
    
    for p in patrones:
        if re.search(p,text) !=None:
            bandera=1
            break
            
    return bandera        
            

In [51]:
# Aplicando la funcion para verificar cuales oraciones tienen iniciativas del tipo conferencia, plan y acuerdo
# Creamos una columna en donde esto se especifique

df2['Tiene_Iniciativa']=df2['Oracion'].apply(oracion_iniciativa)
df2


,Oracion,Anio,Longitud,Tiene_Iniciativa
0,"Had the United Nations not been faced in its five years of existence with countless obstacles, had it not been called upon to deal with so many alarming symptoms in the historical development of a convalescent world, it would be difficult to justify its creation, to welcome its birth and to hang on it, as men hoist a banner to a masthead, the hopes of peace and security of a profoundly disturbed epoch",1950,73,0
1,"The fact is that the struggle which this young yet already sorely tried Organization has waged, and is still waging, is the best proof of the timeliness of its conception and creation, and of the urgent and imperious need for the work it is doing today",1950,46,0
2,Certainly this has not been merely another episode in the political history of the peoples,1950,15,0
3,"The recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement",1950,23,1
4,"That is why the United Nations (united for agreement, concord and peace) neither can nor should be regarded as one of those familiar panaceas created for interested motives in international political laboratories with the object of providing temporary compromise solutions for more or less selfish problems; it must be recognized for what it is, a sacred idea conceived by the peoples at a time of grave and legitimate struggle for the permanent protection of their democratic existence, autonomy, dignity, peace and security",1950,82,1
...,...,...,...,...
5651,We find that the people are free,2014,7,0
5652,They are free to know which parts of their past they wish to use as a guide to their future and which they wish to leave aside,2014,27,0
5653,"We will find a way to work together, as we have done with the neighbours with whom we share an island, or as we found yesterday, during the climate summit, with the other countries with which we share the planet",2014,40,0
5654,"The citizens that we represent, increasingly better informed, demand that we live up to our responsibility and our declared commitment: to preserve peace, to promote development and to make education and health care the right of all inhabitants of the planet",2014,41,0


In [52]:
# Contando cuantas oraciones tienen iniciativas
count=0
for i in range(len(df2)):
    if df2.loc[i,'Tiene_Iniciativa']==1:
        print(df2.loc[i,'Oracion'])
        print("\n")
        count+=1
print(count)    

 The recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement


 That is why the United Nations (united for agreement, concord and peace) neither can nor should be regarded as one of those familiar panaceas created for interested motives in international political laboratories with the object of providing temporary compromise solutions for more or less selfish problems; it must be recognized for what it is, a sacred idea conceived by the peoples at a time of grave and legitimate struggle for the permanent protection of their democratic existence, autonomy, dignity, peace and security


These two considerations are, first, the sincere desire to point out — so that it may be taken into consideration — that there has been a change in world public opinion in favour of such relations since resolution (I) was adopted on December  and, secondly, the fervent wish to contribute to this world Organ

In [133]:
# Funcion para extraer esas iniciativas
def extraer_iniciativas(text, check):
    
    iniciativas=[]
    
    doc = nlp(text)
    
    # Andamos buscando palabras que hablen de iniciativas
    
    lista_palabras_iniciativas = ['agreement','plan','conference']
    
    # Especificamos los patrones para machear los nombres de las iniciativas
    
    # ESTE PATRON SOLO ME ESTA SACANDO AGREEMENTS Y PLANS
    
    patron=[{'POS':'PROPN','DEP':'compound'},
            {'LOWER':{'IN':lista_palabras_iniciativas},'OP':'+'}
           ]
    
    if (check==0):
        return iniciativas

    # Llamar al constructor del matcher y a decirle que machee

    matcher=Matcher(nlp.vocab)
    matcher.add("matching", [patron])
    emparejamiento = matcher(doc)
    

    for i in range(0,len(emparejamiento)):

        start, end = emparejamiento[i][1], emparejamiento[i][2]
        
        # Cadena emparejada

        cadena=str(doc[start:end])     
            
        iniciativas.append(cadena)     
        
    return iniciativas



In [134]:
# Creando columna para que coloque las iniciativas que encuentra en el texto que macheen el patron
df2['Iniciativas']=df2.apply(lambda x:extraer_iniciativas(x.Oracion,x.Tiene_Iniciativa), axis=1)

In [135]:
# IMPRIMIR LAS FILAS COMPLETAS DEL DATAFRAME EN DONDE SI FUE CAPAZ DE EXTRAER UNA INICIATIVA
for i in range(len(df2)):
    if df2.loc[i,'Tiene_Iniciativa']==1:
         print(df2.loc[i,'Iniciativas'])

[]
[]
[]
[]
[]
[]
[]
[]
[]
['Armistice Agreement']
[]
['Armistice Agreement']
[]
['Brady plan']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['International Plan']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [224]:
# PRUEBEN A CREAR FUNCIONES PARA EXTRAER OTRAS ENTIDADES QUE NO SEAN INICIATIVAS

### Tarea 2: Extraccion de KPES

In [149]:
# Vamos a tomar solo las primeras 4000 oraciones, ya que la funcion que convierte a documento de spacy no permite 
# mas de 1000000 de caracteres para el documento.
oraciones_sample=df2['Oracion'][0:4000]

In [150]:
all_words = ' '.join([word for word in oraciones_sample])

In [151]:
all_words

"Had the United Nations not been faced in its five years of existence with countless obstacles, had it not been called upon to deal with so many alarming symptoms in the historical development of a convalescent world, it would be difficult to justify its creation, to welcome its birth and to hang on it, as men hoist a banner to a masthead, the hopes of peace and security of a profoundly disturbed epoch  The fact is that the struggle which this young yet already sorely tried Organization has waged, and is still waging, is the best proof of the timeliness of its conception and creation, and of the urgent and imperious need for the work it is doing today Certainly this has not been merely another episode in the political history of the peoples  The recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement  That is why the United Nations (united for agreement, concord and peace) neither can no

In [152]:
all_words=all_words.lower()

In [153]:
all_words

"had the united nations not been faced in its five years of existence with countless obstacles, had it not been called upon to deal with so many alarming symptoms in the historical development of a convalescent world, it would be difficult to justify its creation, to welcome its birth and to hang on it, as men hoist a banner to a masthead, the hopes of peace and security of a profoundly disturbed epoch  the fact is that the struggle which this young yet already sorely tried organization has waged, and is still waging, is the best proof of the timeliness of its conception and creation, and of the urgent and imperious need for the work it is doing today certainly this has not been merely another episode in the political history of the peoples  the recent wars, with their unparalleled violence and enormous destructive power, engendered in the peoples a boundless desire for peace, concord and agreement  that is why the united nations (united for agreement, concord and peace) neither can no

In [154]:
# Voy a convertir el string con todos los speeches a un formato de documento spacy
speeches=textacy.make_spacy_doc(all_words, lang='en_core_web_sm')

In [157]:
# Con este linea pedimos que se obtengan los mejores N KPEs desde el texto
textacy.extract.keyterms.textrank(speeches,topn=10)
# INTENTAR CONFIGURAR EL METODO PARA QUE EXTRAIGA KPEs DE UNA LONGITUD ESPECIFICA

[('new international economic order', 0.005861079940724801),
 ('avertanother nuclear holocaust general assembly d plenary meetingfiftieth session october small country',
  0.005661248415778),
 ('work ofthe united nations international research', 0.005421872191519489),
 ('present international economic crisis', 0.005197459204647488),
 ('critical international economic situation', 0.005020882248912078),
 ('country international tension', 0.004881901965174675),
 ('new international sugar agreement', 0.004725790012666686),
 ('international economic cooperation', 0.004704406766109519),
 ('international economic relation', 0.004659735625768586),
 ('financial assistance todeveloping country human resource',
  0.004631342663065059)]

### Tarea 3: Encontrando NERs con modelo preentrenado.

In [209]:
#!"/Users/lbeato/anaconda3/bin/python"  -m spacy download en_core_web_trf
#!"/Users/lbeato/anaconda3/bin/python"  -m spacy download  en_core_web_lg

In [220]:
nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load('en_core_web_trf')
#nlp = spacy.load('en_core_web_lg')

In [221]:
entidades=nlp(all_words).ents
entidades

(the united nations,
 five years,
 today,
 concord,
 the united nations,
 concord,
 one,
 the united nations,
 one,
 the democratic republic of south korea,
 armed force marks,
 zero,
 the dominican republic,
 south korea,
 democratic,
 the united nations,
 united action,
 the united nations,
 american,
 the united nations,
 first,
 the dominican republic,
 united nations,
 first,
 the united nations,
 the united nations,
 spain,
 fifth,
 the dominican republic,
 a/,
 the dominican republic,
 some twenty,
 the dominican republic,
 two,
 first,
 the united nations,
 spain,
 two,
 first,
 december,
 secondly,
 a quarter,
 the united nations,
 dominican,
 fernanda,
 the general assembly,
 the dominican republic,
 next year,
 the dominican republic,
 first,
 dominican,
 the united nations,
 latin american,
 caribbean,
 latin america,
 caribbean,
 one,
 millions,
 democratic,
 more than seven decades ago,
 the united nations,
 latin american,
 caribbean,
 two,
 first,
 two,
 thousands,
 dec

In [223]:
{(ent.text.strip(), ent.label_) for ent in nlp(all_words).ents}

{('#', 'CARDINAL'),
 ('$', 'CARDINAL'),
 ('$billion', 'MONEY'),
 ('$billion to $billion', 'MONEY'),
 ('$per metric', 'MONEY'),
 ('$to $', 'MONEY'),
 ('$to $per metric tonper cent', 'MONEY'),
 ('1of', 'ORDINAL'),
 ('4september', 'CARDINAL'),
 ('a century', 'DATE'),
 ('a day', 'DATE'),
 ('a few centuries later', 'DATE'),
 ('a few day ago', 'DATE'),
 ('a few hours', 'TIME'),
 ('a few months ago', 'DATE'),
 ('a few weeks', 'DATE'),
 ('a few years', 'DATE'),
 ('a few years ago', 'DATE'),
 ('a little time ago', 'DATE'),
 ('a metric ton', 'QUANTITY'),
 ('a minute', 'TIME'),
 ('a new century', 'DATE'),
 ('a new decade', 'DATE'),
 ('a new year', 'DATE'),
 ('a number of years', 'DATE'),
 ('a quarter', 'DATE'),
 ('a quarter of a century', 'DATE'),
 ('a united nations', 'ORG'),
 ('a whole decade', 'DATE'),
 ('a year', 'DATE'),
 ('a/', 'LANGUAGE'),
 ('a/', 'NORP'),
 ('a/', 'PRODUCT'),
 ('abandon centuriesold practicesa', 'ORG'),
 ('abc', 'ORG'),
 ('abdelaziz', 'PERSON'),
 ('about $million', 'MONEY'

In [ ]:
# PRUEBEN CON OTROS MODELOS DE LENGUAJE A VER QUE SALE

### Tarea 4: Encontrando NERs entrenando con mis propios datos en Spacy.

In [19]:
# Funcion que lee el archivo NER, devolviendo solo los tokens

def read_file(file_path):
    file_path = Path(file_path)
    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    for doc in raw_docs: 
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token = line.split('\t')
            tokens.append(token[0])
        cadena = ' '.join(tokens);
        token_docs.append(cadena)
    return token_docs

In [20]:
# Funcion que lee el archivo NER, devolviendo solo los tipos de entidad

def read_entidades(file_path):
    file_path = Path(file_path)
    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    tags = []
    for doc in raw_docs: 
        for line in doc.split('\n'):
            l = line.split('\t')
            tags.append(l[1])
    return tags

In [26]:
# Leer el corpus de train de defense

docs=read_file('defense_train.txt')
len(docs)

765

In [29]:
pprint(docs)

['Operation Steel Curtain ( Arabic : ا ل ح ج ا ب ا ل ف و ل ا ذ ي Al Hejab '
 'Elfulathi ) was a military operation executed by coalition forces in early '
 'November 2005 to reduce the flow of foreign insurgents crossing the border '
 'and joining the Iraqi insurgency .',
 'The hospital has facilities for MRI and CT scan , a gamma camera and '
 'lithotripsy device .',
 'The operation was important in that it was the first large scale deployment '
 'of the New Iraqi Army .',
 'This was my first visit to Uzbekistan and an important opportunity to deepen '
 'and strengthen bilateral relations between the UK and Uzbekistan on a number '
 'of issues important to us both .',
 'The group was founded by Sheikh Abu Omar al - Ansari . "',
 'And hopefully , by reading , we can achieve this . " " "',
 'But nobody can leave now .',
 'The Members of the Security Council are encouraged by the decision of Iraqi '
 'President Fuad Masum to nominate a new Prime Minister - designate .',
 'Iraq',
 'We ext

In [36]:
ent = read_entidades('defense_train.txt')
print(set(ent))

{'I-Weapon', 'I-Nationality', 'I-Organisation', 'I-Location', 'I-Temporal', 'B-Organisation', 'I-MilitaryPlatform', 'B-Weapon', 'B-Temporal', 'O', 'B-Nationality', 'B-Money', 'B-MilitaryPlatform', 'I-Quantity', 'I-Money', 'B-DocumentReference', 'I-DocumentReference', 'B-Location', 'I-Person', 'B-Quantity', 'B-Person'}


In [37]:
# Donde esta el ejecutable de python
print(sys.executable)

/Users/lbeato/anaconda3/bin/python


In [39]:
# Aqui se crea una carpeta para colocar los documentos de extension spacy que seran usados en los pipelines
#os.mkdir('spacyNER_data')

In [40]:
# Convertir mis datasets a documentos de Spacy

!"/Users/lbeato/anaconda3/bin/python" -m spacy convert "defense_train.txt" spacyNER_data -c ner
!"/Users/lbeato/anaconda3/bin/python" -m spacy convert "defense_test.txt" spacyNER_data -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (765 documents):
spacyNER_data/defense_train.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (200 documents):
spacyNER_data/defense_test.spacy


In [41]:
# Cargando el archivo de configuracion de spacy para el pipeline

!"/Users/lbeato/anaconda3/bin/python" -m spacy init fill-config data/base_config.cfg data/config.cfg

✔ Auto-filled config with all values
✔ Saved config
data/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [47]:
# Entrenando mi modelo con los datos de entrenamiento de defensa (el modelo se guardara en una carpeta llamada output_folder)
!"/Users/lbeato/anaconda3/bin/python" -m spacy train data/config.cfg --paths.train spacyNER_data/defense_train.spacy --paths.dev spacyNER_data/defense_train.spacy --output output_folder


ℹ Saving to output directory: output_folder
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-06-20 11:26:27,208] [INFO] Set up nlp object from config
[2023-06-20 11:26:27,215] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-20 11:26:27,217] [INFO] Created vocabulary
[2023-06-20 11:26:27,217] [INFO] Finished initializing nlp object
[2023-06-20 11:26:28,146] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     53.00    3.71    2.11   15.19    0.04
  0     200        433.54   4715.34   37.60   46.08   31.75    0.38
  2     400        551.33   3747.11   62.13   68.30   56.99    0.62
  3     600        688.39   3788.24 

In [45]:
# Creando carpeta para guardar resultados de la prueba
os.mkdir('result')

In [49]:
# Probando mi modelo con los datos de prueba de defensa
!"/Users/lbeato/anaconda3/bin/python" -m spacy evaluate output_folder/model-best SpacyNER_data/defense_test.spacy -dp result

ℹ Using CPU

================================== Results ==================================

TOK     -    
NER P   59.49
NER R   62.13
NER F   60.78
SPEED   17939


=============================== NER (per type) ===============================

                        P       R       F
Person              73.27   69.81   71.50
Temporal            64.58   67.39   65.96
Organisation        61.81   62.68   62.24
Location            56.91   66.45   61.31
Quantity            40.48   56.67   47.22
MilitaryPlatform     0.00    0.00    0.00
Weapon              43.75   41.18   42.42
DocumentReference   54.55   75.00   63.16
Money                0.00    0.00    0.00
Nationality         57.14   40.00   47.06

/Users/lbeato/anaconda3/lib/python3.10/site-packages/spacy/displacy/__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` p

### Tarea 5: Encontrando NERs entrenando un algoritmo CRF

In [56]:
# Funcion que construye las caracteristicas a tomar en cuenta por el algoritmo CRF.
# Se toma una ventana de 2 palabras a cada lado y las etiquetas pos de cada una de ellas.

def sent2feats(sentence):
    feats = []
    sen_tags = pos_tag(sentence) 
    for i in range(0,len(sentence)):
        word = sentence[i]
        wordfeats = {}
        wordfeats['word'] = word
        if i == 0:
            wordfeats["prevWord"] = wordfeats["prevSecondWord"] = "<S>"
        elif i==1:
            wordfeats["prevWord"] = sentence[0]
            wordfeats["prevSecondWord"] = "</S>"
        else:
            wordfeats["prevWord"] = sentence[i-1]
            wordfeats["prevSecondWord"] = sentence[i-2]
        #next two words as features
        if i == len(sentence)-2:
            wordfeats["nextWord"] = sentence[i+1]
            wordfeats["nextNextWord"] = "</S>"
        elif i==len(sentence)-1:
            wordfeats["nextWord"] = "</S>"
            wordfeats["nextNextWord"] = "</S>"
        else:
            wordfeats["nextWord"] = sentence[i+1]
            wordfeats["nextNextWord"] = sentence[i+2]
        
        wordfeats['tag'] = sen_tags[i][1]
        if i == 0:
            wordfeats["prevTag"] = wordfeats["prevSecondTag"] = "<S>"
        elif i == 1:
            wordfeats["prevTag"] = sen_tags[0][1]
            wordfeats["prevSecondTag"] = "</S>"
        else:
            wordfeats["prevTag"] = sen_tags[i - 1][1]

            wordfeats["prevSecondTag"] = sen_tags[i - 2][1]
            # next two words as features
        if i == len(sentence) - 2:
            wordfeats["nextTag"] = sen_tags[i + 1][1]
            wordfeats["nextNextTag"] = "</S>"
        elif i == len(sentence) - 1:
            wordfeats["nextTag"] = "</S>"
            wordfeats["nextNextTag"] = "</S>"
        else:
            wordfeats["nextTag"] = sen_tags[i + 1][1]
            wordfeats["nextNextTag"] = sen_tags[i + 2][1]
        # Aqui usted puede aniadir mas features si lo desea
        feats.append(wordfeats)
    return feats

# Probando lo que hace esta funcion que crea  los features que se usaran como secuencia
print(sent2feats(['John', 'is', 'working', 'at', 'a', 'laboratory']))

[{'word': 'John', 'prevWord': '<S>', 'prevSecondWord': '<S>', 'nextWord': 'is', 'nextNextWord': 'working', 'tag': 'NNP', 'prevTag': '<S>', 'prevSecondTag': '<S>', 'nextTag': 'VBZ', 'nextNextTag': 'VBG'}, {'word': 'is', 'prevWord': 'John', 'prevSecondWord': '</S>', 'nextWord': 'working', 'nextNextWord': 'at', 'tag': 'VBZ', 'prevTag': 'NNP', 'prevSecondTag': '</S>', 'nextTag': 'VBG', 'nextNextTag': 'IN'}, {'word': 'working', 'prevWord': 'is', 'prevSecondWord': 'John', 'nextWord': 'at', 'nextNextWord': 'a', 'tag': 'VBG', 'prevTag': 'VBZ', 'prevSecondTag': 'NNP', 'nextTag': 'IN', 'nextNextTag': 'DT'}, {'word': 'at', 'prevWord': 'working', 'prevSecondWord': 'is', 'nextWord': 'a', 'nextNextWord': 'laboratory', 'tag': 'IN', 'prevTag': 'VBG', 'prevSecondTag': 'VBZ', 'nextTag': 'DT', 'nextNextTag': 'NN'}, {'word': 'a', 'prevWord': 'at', 'prevSecondWord': 'working', 'nextWord': 'laboratory', 'nextNextWord': '</S>', 'tag': 'DT', 'prevTag': 'IN', 'prevSecondTag': 'VBG', 'nextTag': 'NN', 'nextNextT

In [59]:
# Lee archivo en formato conll, devolviendo una listas con las palabra y el tipo de entidad

def load_data_conll(file_path):
    myoutput,words,tags = [],[],[]
    fh = open(file_path)
    for line in fh:
        line = line.strip() 
        if "\t" not in line:
            myoutput.append([words,tags])
            words,tags = [],[]
        else:
            word, tag = line.split("\t")
            words.append(word)
            tags.append(tag)
    fh.close()
    return myoutput

In [60]:
# Función para extraer los features de los datos en formato conll, después de descargarlos.
def get_feats_conll(conll_data):
    feats = []
    labels = []
    for sentence in conll_data:
        feats.append(sent2feats(sentence[0]))
        labels.append(sentence[1])
    return feats, labels

In [63]:
# Funcion para imprimir la matriz de confusion, solo para las etiquetas que se estan considerando (Person, Organisation y Location)
def print_cm(cm, labels):
    print("\n")
    columnwidth = 5
    empty_cell = " " * columnwidth
    # Imprimir encabezado
    print("    " + empty_cell, end=" ")
    for label in labels:
        if label in ('B-Person', 'I-Person', 'B-Organisation', 'I-Organisation', 'B-Location', 'I-Location'):
           print("%{0}s".format(columnwidth) % label[0:3], end=" ")
    # Imprimir filas
    print('\n')
    for i, label1 in enumerate(labels):
        if label1 in ('B-Person', 'I-Person', 'B-Organisation', 'I-Organisation', 'B-Location', 'I-Location'):
            print("    %{0}s".format(columnwidth) % label1[0:3], end=" ")
            sum = 0
            for j in [3,4,11,12,13,14]: # Posiciones de mi matriz para las etiquetas que me interesan
                cell = "%{0}.0f".format(columnwidth) % cm[i, j]
                sum =  sum + int(cell)
                print(cell, end=" ")
            print('\n')     

In [73]:
# Funcion para obtener la matriz de confusion

def get_confusion_matrix(y_true,y_pred,labels):
    trues,preds = [], []
    for yseq_true, yseq_pred in zip(y_true, y_pred):
        trues.extend(yseq_true)
        preds.extend(yseq_pred)
    print_cm(confusion_matrix(trues,preds,labels=labels),labels)

In [74]:
# Entrenando un algoritmo CRF

def train_seq(X_train,Y_train,X_dev,Y_dev):
    crf = CRF(algorithm='lbfgs', c1=0.1, c2=10, max_iterations=50)#, all_possible_states=True)
    # Entrenando
    crf.fit(X_train, Y_train)
    labels = list(crf.classes_)
    #Probando
    y_pred = crf.predict(X_dev)
    sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
    #Obteniendo matriz de confusion y metricas
    get_confusion_matrix(Y_dev, y_pred,labels=sorted_labels)
 

In [75]:
# Ejecutando las funciones que construyen el modelo y muestran metricas de desempenio

# Cargando los datos de training y testing

train_path = 'defense_train.txt'
test_path = 'defense_test.txt'

data_train = load_data_conll(train_path)
data_test = load_data_conll(test_path)

#pprint(data_train)
# Modelando e imprimiendo resultados

print("Entrenando el modelo de clasificacion secuencial CRF")
feats, labels = get_feats_conll(data_train)
testfeats, testlabels = get_feats_conll(data_test)
train_seq(feats, labels, testfeats, testlabels)
print("\nModelo secuencial finalizado")

Entrenando el modelo de clasificacion secuencial CRF


            B-L   I-L   B-O   I-O   B-P   I-P 

      B-L    59     4    12     3     3     0 

      I-L     4    62     1    21     0     0 

      B-O     4     1   131    13     0     0 

      I-O     3     2     9   246     1     3 

      B-P     4     1     7     1    41     4 

      I-P     0     3     2    15     1    69 


Modelo secuencial finalizado
